In [1]:
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [2]:
uri = "bolt://localhost:7687/"
user = "neo4j"
password = "password"

In [3]:
llm = ChatOpenAI(
    model="ai/phi4:latest",
    base_url="http://127.0.0.1:12434/engines/v1",
    api_key="ignored",
    temperature=0
)

In [4]:
graph = Neo4jGraph(url=uri,username=user, password=password, database='brainwave')

In [116]:
prompt = ChatPromptTemplate.from_template("""
You are an expert Neo4j developer. Use the following database schema to write
a Cypher statement to answer the user's question. Only provide the query with NO additional text. 
                                          
Schema: {schema}
             
Question: {question}""", partial_variables={'schema':graph.schema})

In [117]:
cypher_chain = prompt | llm | (lambda msg: msg.content) | StrOutputParser() | (lambda s: s.replace("cypher\n","").replace("```",""))

In [170]:
QA_PROMPT = """You are a helpful assistant. Use the data retrieved from the graph to answer the user's question. Data: "{context}" 
Do not worry about confirming the information returned from the graph or providing any additional context. Always provide the answer returned by the graph. Do not provide any additional knowledge from your knowledge bank.
"""

In [171]:
answer_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(QA_PROMPT),
    HumanMessagePromptTemplate.from_template("Question: {question}")
])

In [172]:
qa_chain = {
    "context": cypher_chain | RunnableLambda(lambda cypher: graph.query(cypher)),
    "question": RunnablePassthrough()
}    | answer_prompt | llm | StrOutputParser()

In [173]:
qa_chain.invoke({"question":"What is the date of birth and last name for the user of phone 84795203656?"})

'The date of birth is 1991-03-16 and the last name is Marisa.'

In [174]:
qa_chain.invoke({"question":"What is the capitol of Spain"})

"I'm sorry, but I don't have access to the data needed to answer your question. Please refer to a reliable source for the capital of Spain."